In [ ]:
# Importing libraries:

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Importing the os library specifically for reading the csv once I've created the file in my working directory.
import os

# web-based requests
import requests

In [ ]:
base_url = 'https://python.zach.lol'
print(requests.get(base_url).text)

In [ ]:
response = requests.get(base_url + '/documentation')
print(response.json()['payload'])

In [ ]:
# Now I have the info I need to start getting specific portions of the information via the API.

## Items Data

### 1. Using the code from the lesson as a guide, create a dataframe named items that has all of the data for items.

In [ ]:
response = requests.get('https://python.zach.lol/api/v1/items')

data = response.json()
data.keys()

In [ ]:
data['payload'].keys()

In [ ]:
print('max_page: %s' % data['payload']['max_page'])
print('next_page: %s' % data['payload']['next_page'])

In [ ]:
data['payload']['items'][:2]

In [ ]:
# So I need to go back and recall how to work with dictionaries. I need to use those techniques to parse the info, instead of trying to use API to prep the data.

In [ ]:
# data from page 1

items = pd.DataFrame(data['payload']['items'])
print(items.shape)
items.head()

In [ ]:
data['payload']['items']

In [ ]:
data['payload']['next_page']

In [ ]:
response = requests.get(base_url + data['payload']['next_page'])
data = response.json()

print('max_page: %s' % data['payload']['max_page'])
print('next_page: %s' % data['payload']['next_page'])

items = pd.concat([items, pd.DataFrame(data['payload']['items'])]).reset_index()

In [ ]:
items.head()

In [ ]:
response = requests.get(base_url + data['payload']['next_page'])
data = response.json()

print('max_page: %s' % data['payload']['max_page'])
print('next_page: %s' % data['payload']['next_page'])

items = pd.concat([items, pd.DataFrame(data['payload']['items'])]).reset_index()

In [ ]:
items.head()

In [ ]:
items.drop(columns = ['level_0', 'index'], inplace = True)
items.shape

In [ ]:
items.head()

## Store Data

### 2. Do the same thing, but for stores.



In [ ]:
response_stores = requests.get('https://python.zach.lol/api/v1/stores')

data_stores = response_stores.json()
data_stores.keys()

In [ ]:
data_stores['payload'].keys()

In [ ]:
print('max_page: %s' % data_stores['payload']['max_page'])
print('next_page: %s' % data_stores['payload']['next_page'])

In [ ]:
data_stores['payload'].keys()
data_stores['payload']['stores'][:2]

In [ ]:
stores_df = pd.DataFrame(data_stores['payload']['stores'])
print(stores_df.shape)
stores_df

In [ ]:
# Only one page of store data, so I don't need to concatinate multiple pages here.

## Sales Data

### 3. Extract the data for sales. 

- There are a lot of pages of data here, so your code will need to be a little more complex. Your code should continue fetching data from the next page until all of the data is extracted.

In [ ]:
base_url_sales = 'https://python.zach.lol'

In [ ]:
# Create a function from what I'm doing here. While loop is based on next_page != "None"

response_sales = requests.get('https://python.zach.lol/api/v1/sales')

data_sales = response_sales.json()
data_sales.keys()

In [ ]:
data_sales['payload'].keys()

In [ ]:
print('max_page: %s' % data_sales['payload']['max_page'])
print('next_page: %s' % data_sales['payload']['next_page'])

In [ ]:
# This code is looking at a set number of entries in the dictionary I'm calling from the api:
data_sales['payload']['sales'][:2]

In [ ]:
sales_df = pd.DataFrame(data_sales['payload']['sales'])
sales_df.head()

In [ ]:
sales_df.shape

In [ ]:
response_sales = requests.get(base_url + data_sales['payload']['next_page'])
data_sales = response_sales.json()

print('max_page: %s' % data_sales['payload']['max_page'])
print('next_page: %s' % data_sales['payload']['next_page'])

sales_df = pd.concat([sales_df, pd.DataFrame(data_sales['payload']['sales'])]).reset_index()
sales_df.shape

In [ ]:
# Calling the same thing, but now I'm calling as many items are on the first page:
len(data_sales['payload']['sales'])
# There are 5000 items per page it seems. So the number of rows I'll have is:

row_total_guess = len(data_sales['payload']['sales']) * (data_sales['payload']['max_page'])
print(f'The estimated total number of rows of the combined sales dataframe is {row_total_guess:,}.')

In [ ]:
def get_sales(base_url):
    
    response = requests.get('https://python.zach.lol/api/v1/sales')
    data = response.json()
    data.keys()
    print('max_page: %s' % data['payload']['max_page'])
    print('next_page: %s' % data['payload']['next_page'])
    
    df_sales = pd.DataFrame(data['payload']['sales'])
    
    while data['payload']['next_page'] != "None":
        response = requests.get(base_url + data['payload']['next_page'])
        data = response.json()
        
        df_sales = pd.concat([df_sales, pd.DataFrame(data['payload']['sales'])])
        
        if data['payload']['next_page'] == None:
            break
            
    df_sales = df_sales.reset_index()
    print('full_shape', df_sales.shape)
    return df_sales
    
    

In [ ]:
sales_df = get_sales(base_url_sales)

In [ ]:
sales_df.shape

In [ ]:
sales_df

In [ ]:
sales_df.drop(columns = 'index', inplace = True)

In [83]:
sales_df.head()

,item,sale_amount,sale_date,sale_id,store
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1


### 4. Save the data in your files to local csv files so that it will be faster to access in the future.

In [84]:
# Writing to a csv:

def write_csv(df, csv_name):
    df.to_csv(csv_name, index = False)
    print('Completed writing df to .csv file')
    

In [85]:
write_csv(sales_df, 'sales_df.csv')

Completed writing df to .csv file


In [86]:
write_csv(stores_df, 'stores_df.csv')

Completed writing df to .csv file


In [87]:
write_csv(items, 'items_df.csv')

Completed writing df to .csv file


### 5. Combine the data from your three separate dataframes into one large dataframe.

In [88]:
print(sales_df.shape)
sales_df.head()

(913000, 5)


,item,sale_amount,sale_date,sale_id,store
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1


In [89]:
print(stores_df.shape)
stores_df.head()

(10, 5)


,store_address,store_city,store,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218


In [90]:
print(items.shape)
items.head()

(20, 6)


,item_brand,item,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


In [91]:
# so, sales_df has both of the ids needed to join the two tables. I'll need to use two joins, both of them (I believe) will be a left join.

sales_test = sales_df.copy()

In [92]:
item_test = items.copy()
item_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   item_brand  20 non-null     object 
 1   item        20 non-null     int64  
 2   item_name   20 non-null     object 
 3   item_price  20 non-null     float64
 4   item_upc12  20 non-null     object 
 5   item_upc14  20 non-null     object 
dtypes: float64(1), int64(1), object(4)
memory usage: 1.1+ KB


In [113]:
items.rename(columns = {'item_id': 'item'}, inplace = True)
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   item_brand  20 non-null     object 
 1   item        20 non-null     int64  
 2   item_name   20 non-null     object 
 3   item_price  20 non-null     float64
 4   item_upc12  20 non-null     object 
 5   item_upc14  20 non-null     object 
dtypes: float64(1), int64(1), object(4)
memory usage: 1.1+ KB


In [94]:
stores_df.rename(columns = {'store_id': 'store'}, inplace = True)
stores_df.head()

,store_address,store_city,store,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218


In [114]:
items.head()

,item_brand,item,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


In [115]:
sales_df.head()

,item,sale_amount,sale_date,sale_id,store
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1


In [117]:
left_merge = pd.merge(sales_df, items, how = 'left', on = 'item')
left_merge.head()

,item,sale_amount,sale_date,sale_id,store,item_brand,item_name,item_price,item_upc12,item_upc14
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013


In [118]:
left_merge.shape

(913000, 10)

In [119]:
all_df = pd.merge(left_merge, stores_df, how = 'left', on = 'store')
all_df.head()

,item,sale_amount,sale_date,sale_id,store,item_brand,item_name,item_price,item_upc12,item_upc14,store_address,store_city,store_state,store_zipcode
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253


In [120]:
all_df.shape

(913000, 14)

In [121]:
# Saving to csv:

write_csv(all_df, 'add_df.csv')

Completed writing df to .csv file
